## Import libraries

In [1]:
import dash # initialize your application
import dash_core_components as dcc # allows you to create interactive components like graphs, dropdowns, or date ranges
import dash_html_components as html # access HTML tags
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from datetime import datetime
from dash.dependencies import Output, Input

## Import data

In [16]:
data = pd.read_csv("Swim_dataset.csv") # read avocado csv file

# filter the unnecessary columns
unnec_cols = data.columns[10:]
data = data.drop(columns=unnec_cols)

# rename "Unnamed: 0" column name to "Date"
data = data.rename(columns={"Unnamed: 0":"Date"})

# convert the contents in the date column into a datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d") # convert the date into a date format

# set Date as an index 
# data = data.set_index('Date')

# convert nan to 0
data = data.fillna(0)

data

,Date,Warm Up,Kick,Pull,Swim,Drill,Main Set,Post Set,Cool Down,Total
0,2020-01-02,400.0,600.0,0.0,600.0,400.0,2800.0,100.0,400.0,5300
1,2020-01-03,400.0,0.0,0.0,1600.0,400.0,2000.0,600.0,400.0,5400
2,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2020-01-06,800.0,0.0,0.0,2200.0,0.0,2500.0,880.0,200.0,6580
...,...,...,...,...,...,...,...,...,...,...
360,2020-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
361,2020-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
362,2020-12-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
363,2020-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


### Remove rows with all zeros

In [18]:
column_list = data.columns[1:]
data = data[(data[column_list].T != 0.0).any()]
data

,Date,Warm Up,Kick,Pull,Swim,Drill,Main Set,Post Set,Cool Down,Total
0,2020-01-02,400.0,600.0,0.0,600.0,400.0,2800.0,100.0,400.0,5300
1,2020-01-03,400.0,0.0,0.0,1600.0,400.0,2000.0,600.0,400.0,5400
4,2020-01-06,800.0,0.0,0.0,2200.0,0.0,2500.0,880.0,200.0,6580
5,2020-01-07,800.0,0.0,0.0,920.0,0.0,3000.0,0.0,400.0,5120
6,2020-01-08,800.0,0.0,0.0,1000.0,0.0,4500.0,0.0,400.0,6700
...,...,...,...,...,...,...,...,...,...,...
210,2020-07-30,1000.0,700.0,1800.0,1900.0,0.0,800.0,0.0,200.0,6400
211,2020-07-31,600.0,1100.0,2000.0,600.0,0.0,3000.0,100.0,200.0,7600
212,2020-08-01,800.0,600.0,600.0,3600.0,400.0,750.0,100.0,200.0,7050
214,2020-08-03,600.0,300.0,3000.0,600.0,200.0,3000.0,200.0,200.0,8100


#### 1 day

In [19]:
# 1일치
one_day_data = data.iloc[0][1:-1]

# columns
categories = data.columns[1:-1]

## Initialize the application

In [20]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "Swimming Training Time Analysis"

## App layout

In [21]:
graph_bgcolor = "#0A165A"
font_color = "#FFFFFF"

In [32]:
app.layout = html.Div(
    children=[
        
        # heading
        html.Div(
            children=[
                html.P(
                    children="🏊‍♂️", className="header-emoji"
                ),
                html.H1(
                    children="Swimming Training Analysis", className="header-title"
                ),
                html.P(
                    children="Analyze the how the training time affects the result",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        
        # Graph part
        html.Div(
            children=[
                
                # Competition graph
                html.Div(
                    children=[
                        
                        # title
                        html.Div(children='Competition Graph', className='competition-title'),
                        
                        # menu
                        html.Div(
                            children=[
                                
                                # average-total selection
                                dcc.RadioItems(
                                    id='competition-avg-total-selector',
                                    options=[
                                        {'label': 'Average', 'value': 'avg'},
                                        {'label': 'Total', 'value': 'total'},
                                    ],
                                    value='avg',
                                    labelStyle={
                                        'display': 'flex',
                                        'margin-bottom': '5px',
                                        'font-weight': 300,
                                        'color': '#FFFFFF',
                                    },
                                ),
                                
                                # category selector
                                dcc.Dropdown(
                                    id='competition-category-selector',
                                    options=[
                                        {'label': 'Warm Up', 'value': 'Warm Up'},
                                        {'label': 'Kick', 'value': 'Kick'},
                                        {'label': 'Pull', 'value': 'Pull'},
                                        {'label': 'Swim', 'value': 'Swim'},
                                        {'label': 'Drill', 'value': 'Drill'},
                                        {'label': 'Main Set', 'value': 'Main Set'},
                                        {'label': 'Post Set', 'value': 'Post Set'},
                                        {'label': 'Cool Down', 'value': 'Cool Down'},
                                    ],
                                    multi=True,
                                    style={"min-width": "200px"},
                                ),
                            ],
                            className='menu'
                        ),
                        
                        # graph
                        dcc.Graph(
                            id='competition-graph-id',
                            className='competition-graph',
                            figure={
                                "layout": {
                                    "paper_bgcolor": graph_bgcolor,
                                    "plot_bgcolor": graph_bgcolor,
                                    "font": {
                                        "color": "#FFFFFF"
                                    }
                                },
                            },
                        ),
                    ],
                    className='competition-graph-container'
                ),
                
                
                # Training graph
                html.Div(
                    children=[
                        
                        # categorical bar plot
                        html.Div(
                            children=[
                                # title
                                html.Div(
                                    children=[
                                        html.Div(children='Categorical Graph', className='categorical-graph-title'),
                                        dcc.DatePickerRange(
                                            id='category-date-picker',
                                            min_date_allowed=data.Date.min().date(),
                                            max_date_allowed=data.Date.max().date(),
                                            start_date=data.Date.min().date(),
                                            end_date=data.Date.max().date(),
                                            with_portal=True,
                                            style={'display': 'inline-block'},
                                            display_format='DD-MMM-YY',
                                            className='datepicker',
                                        ),
                                    ], 
                                    className='category-title-container',
                                ),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='category-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='category-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            value=["Warm Up"],
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),
                                    ],
                                    className='train-menu',
                                ),

                                # bar plot
                                html.Div(
                                    children=dcc.Graph(
                                        id='categorical-bar-chart',
                                        style={"height": "100%"},
                                        config={"displayModeBar": True},
                                        figure={
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                    ),
                                    className='bar-categorical-graph'
                                ),
                            ],
                            className='bar-categorical-graph-container'
                        ),
                        
                        # Categorical line plot
                        html.Div(
                            children=[
                                # title
                                html.Div(
                                    children=[
                                        html.Div(children='Time Series Graph', className='time-graph-title'),
                                        html.Div(
                                            children=[
                                                html.Button('D', id='daily-button', className='time-series-button', n_clicks=0),
                                                html.Button('W', id='weekly-button', className='time-series-button', n_clicks=0),
                                                html.Button('M', id='monthly-button', className='time-series-button', n_clicks=0),
                                            ],
                                        ),
                                    ], 
                                    className='time-title-container',
                                ),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='time-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='time-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            value=["Warm Up"],
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),
                                    ],
                                    className='train-menu'
                                ),

                                # line plot
                                html.Div(
                                    children=dcc.Graph(
                                        id='timeseries-line-chart',
                                        style={"height": "100%"},
                                        config={"displayModeBar": True},
                                        figure={
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                    ),
                                    className='time-series-graph'
                                ),
                            ],
                            className='time-series-graph-container',
                        ),
                    ],
                    className='training-graph-container',
                ),
            ],
            className='graph-container'
        ),
        
        # space
        html.Div(style={"height": 80}),
    ],
    className='app-container'
)

## Competition bar chart callback

In [33]:
# @app.callback(
#     Output("competition-graph-id", "figure"),
#     [
#         Input("competition-avg-total-selector", "value"),
#         Input("competition-category-selector", "value"),
#     ],
# )
# def update_bar_chart(avg_or_total, categories):
#     # filter by categories
#     new_data = data[categories]
#     # if new_data is in the form
#     if isinstance(new_data, pd.Series):
#         new_data = new_data.to_frame()

#     # get avg or total data
#     total_data = []
#     if type(categories) == str:
#         category_list = list(new_data[categories])
#         total_value = sum(category_list)
#         total_data.append(total_value)
#     else:
#         for category in categories:
#             category_list = list(new_data[category])
#             total_value = sum(category_list)
#             total_data.append(total_value)
    
#     if avg_or_total == 'avg':
#         for i in range(len(total_data)):
#             total_data[i] /= new_data.shape[0]
            
#     bar_chart_figure = {
#         "data": [
#             {
#                 "x": categories,
#                 "y": total_data,
#                 "type": "bar",
#                 "name": "1 month data",
#             },
#         ],
#         "layout": {
#             "paper_bgcolor": graph_bgcolor,
#             "plot_bgcolor": graph_bgcolor,
#             "font": {
#                 "color": "#FFFFFF"
#             }
#         },
#     }
    
#     return bar_chart_figure

## Catetegorical Graph callback

In [34]:
@app.callback(
    Output("categorical-bar-chart", "figure"),
    [
        Input("category-avg-total-selector", "value"),
        Input("category-category-selector", "value"),
        Input("category-date-picker", "start_date"),
        Input("category-date-picker", "end_date"),
    ],
)
def update_categorical_bar_chart(avg_or_total, categories, start_date, end_date):
    # filter by date
    date_mask = (
        (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    cat_data = data.loc[date_mask, :]

    # filter by categories
    cat_data = cat_data[categories]
    # if new_data is in the form
    if isinstance(cat_data, pd.Series):
        cat_data = cat_data.to_frame()

    # get avg or total data
    total_data = []
    if type(categories) == str:
        category_list = list(cat_data[categories])
        total_value = sum(category_list)
        total_data.append(total_value)
    else:
        for category in categories:
            category_list = list(cat_data[category])
            total_value = sum(category_list)
            total_data.append(total_value)
    
    if avg_or_total == 'avg':
        for i in range(len(total_data)):
            total_data[i] /= cat_data.shape[0]
            
    bar_chart_figure = {
        "data": [
            {
                "x": categories,
                "y": total_data,
                "type": "bar",
                "name": "1 month data",
                "max-width": 50,
            },
        ],
        "layout": {
            "paper_bgcolor": graph_bgcolor,
            "plot_bgcolor": graph_bgcolor,
            "font": {
                "color": "#FFFFFF"
            }
        },
    }
    
    return bar_chart_figure

## Categorical Line callback

In [35]:
@app.callback(
    Output("timeseries-line-chart", "figure"),
    [
        Input("time-avg-total-selector", "value"),
        Input("time-category-selector", "value"),
        Input("time-date-range-selector", "start_date"),
        Input("time-date-range-selector", "end_date"),
    ],
)
def update_categorical_bar_chart(avg_or_total, categories, start_date, end_date):
    # filter by date
    date_mask = (
        (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    time_data = data.loc[date_mask, :]
    time_data_date = data.loc[date_mask, :]

    # filter by categories
    time_data = time_data[categories]
    # if new_data is in the form
    if isinstance(time_data, pd.Series):
        time_data = time_data.to_frame()
            
    # data
    data_list = []
    if type(categories) == str:
        data_dict = {
            "x": time_data_date.Date,
            "y": time_data[categories],
            "type": "lines",
        }
    else:
        for category in categories:
            data_dict = {
                "x": time_data_date.Date,
                "y": time_data[category],
                "type": "lines",
                "name": category,
            }
            data_list.append(data_dict)
    
    line_chart_figure = {
        "data": data_list,
        "layout": {
            "paper_bgcolor": graph_bgcolor,
            "plot_bgcolor": graph_bgcolor,
            "font": {
                "color": "#FFFFFF"
            }
        },
    }
    
    return line_chart_figure

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Sep/2021 22:05:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2021 22:05:48] "GET /assets/style.css?m=1632402332.7244425 HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2021 22:05:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2021 22:05:49] "GET /_dash-dependencies HTTP/1.1" 200 -
